# Exercise 8 - VotingClassifier
Load the MNIST data and split it into a training set, a validation set, and a test set (e.g., use 50,000 instances for training, 10,000 for validation, and 10,000 for testing). Then train various classifiers, such as a Random Forest classifier, an Extra-Trees classifier, and an SVM. Next, try to combine them into an ensemble that outperforms them all on the validation set, using a soft or hard voting classifier. Once you have found one, try it on the test set. How much better does it perform compared to the individual classifiers?

In [21]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.utils import Bunch
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier, StackingClassifier
from sklearn.base import BaseEstimator
from sklearn.neighbors import KNeighborsClassifier

## Loading Dataset

In [2]:
mnist: Bunch = fetch_openml('mnist_784', parser= 'auto', as_frame= False)

X, y = mnist.data, mnist.target

X_train: np.ndarray = X[:50000]
y_train: np.ndarray = y[:50000]

X_valid: np.ndarray = X[50000:60000]
y_valid: np.ndarray = y[50000:60000]

X_test: np.ndarray = X[60000:]
y_test: np.ndarray = y[60000:]

## Training Various Models

In [3]:
rnd_forest = RandomForestClassifier(n_jobs= -1, random_state= 42)
extra_trees = ExtraTreesClassifier(n_jobs= -1, random_state= 42)
knn_clf = KNeighborsClassifier(n_jobs= -1)

In [4]:
estimators: list[BaseEstimator] = [rnd_forest, extra_trees, knn_clf]
    
for clf in estimators:
    print(f'Fitting {clf}')
    clf.fit(X_train, y_train)

Fitting RandomForestClassifier(n_jobs=-1, random_state=42)
Fitting ExtraTreesClassifier(n_jobs=-1, random_state=42)
Fitting KNeighborsClassifier(n_jobs=-1)


In [5]:
[clf.score(X_valid, y_valid) for clf in estimators]

[0.9736, 0.9743, 0.9718]

## Voting Classifier (hard voting)

In [6]:
voting_clf = VotingClassifier(
    estimators= [
        ('rnd_forest', RandomForestClassifier(random_state= 42)),
        ('extra_tress', ExtraTreesClassifier(random_state= 42)),
        ('knn', KNeighborsClassifier())
    ],
    n_jobs= -1
)

In [7]:
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('rnd_forest',
                              RandomForestClassifier(random_state=42)),
                             ('extra_tress',
                              ExtraTreesClassifier(random_state=42)),
                             ('knn', KNeighborsClassifier())],
                 n_jobs=-1)

In [8]:
voting_clf.score(X_valid, y_valid)

0.9761

In [9]:
voting_clf.score(X_test, y_test)

0.9722

## Voting Classifier (soft voting)

In [10]:
voting_clf.voting = 'soft'

In [11]:
voting_clf.score(X_valid, y_valid)

0.9773

In [12]:
voting_clf.score(X_test, y_test)

0.9736

# Exercise - 9
Run the individual classifiers from the previous exercise to make predictions on the validation set, and create a new training set with the resulting predictions: each training instance is a vector containing the set of predictions from all your classifiers for an image, and the target is the image's class. Train a classifier on this new training set. Congratulations, you have just trained a blender, and together with the classifiers they form a stacking ensemble! Now let's evaluate the ensemble on the test set. For each image in the test set, make predictions with all your classifiers, then feed the predictions to the blender to get the ensemble's predictions. How does it compare to the voting classifier you trained earlier? Now try again using a StackingClassifier instead: do you get better performance? If so, why?

In [13]:
rnd_forest_pred = rnd_forest.predict(X_valid)
extra_trees_pred = extra_trees.predict(X_valid)
knn_pred = knn_clf.predict(X_valid)

In [14]:
X_pred_train = np.c_[rnd_forest_pred, extra_trees_pred, knn_pred]

In [15]:
X_pred_train

array([['3', '3', '3'],
       ['8', '8', '8'],
       ['6', '6', '6'],
       ...,
       ['5', '5', '5'],
       ['6', '6', '6'],
       ['8', '8', '8']], dtype=object)

In [17]:
blender = RandomForestClassifier(n_jobs= -1, random_state= 43)
blender.fit(X_pred_train, y_valid)

RandomForestClassifier(n_jobs=-1, random_state=43)

In [18]:
rnd_forest_pred = rnd_forest.predict(X_test)
extra_trees_pred = extra_trees.predict(X_test)
knn_pred = knn_clf.predict(X_test)

X_pred_test = np.c_[rnd_forest_pred, extra_trees_pred, knn_pred]

In [19]:
blender.score(X_pred_test, y_test)

0.9698

## Stacking Classifier

In [22]:
stacking_clf = StackingClassifier(
    estimators= [
        ('rnd_forest', RandomForestClassifier(n_jobs= -1, random_state= 42)),
        ('extra_trees', ExtraTreesClassifier(n_jobs= -1, random_state= 42)),
        ('knn', KNeighborsClassifier(n_jobs= -1))
    ]
)

In [23]:
stacking_clf.fit(X_train, y_train)

StackingClassifier(estimators=[('rnd_forest',
                                RandomForestClassifier(n_jobs=-1,
                                                       random_state=42)),
                               ('extra_trees',
                                ExtraTreesClassifier(n_jobs=-1,
                                                     random_state=42)),
                               ('knn', KNeighborsClassifier(n_jobs=-1))])

In [25]:
stacking_clf.score(X_test, y_test)

0.9772